In [2]:
import numpy as np
from numpy import std
from numpy import mean
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pickle

In [14]:
df = pd.read_csv("sql_df_preprocessed.csv")
df = df.drop(["full_url", "parsed_url", "event_time", "hostname"], axis=1)
df[df.duplicated()].drop_duplicates()
df

,sql_keyword,params_length,request_volume,special_chars,values_to_length,injection
0,8,178,3.5,11,10.674157,1
1,0,5,3.5,1,20.000000,1
2,0,13,3.5,5,38.461538,1
3,0,21,3.5,3,14.285714,1
4,1,24,3.5,3,16.666667,1
...,...,...,...,...,...,...
6126,1,48,0.5,1,4.166667,0
6127,0,34,0.5,1,2.941176,0
6128,0,37,0.5,1,2.702703,0
6129,0,32,0.5,1,3.125000,0


In [4]:
# load data set and split it into metrics and classifier
np_dataset = df.to_numpy()
X = df.iloc[:, :-1].values
y = df. iloc [:, -1].values

print(X.shape)
print(y.shape)

(6131, 5)
(6131,)


In [5]:
test_vector = []
predicted_vector = []

#init cross validation
kfold = KFold(n_splits=4, shuffle=True, random_state=2)

for train, test in kfold.split(np_dataset):
    X_train = np_dataset[train][:, :-1]
    y_train = np_dataset[train][:, -1]

    X_test = np_dataset[test][:, :-1]
    y_test = np_dataset[test][:, -1]

    # create model and train it
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    break

y_pred = knn.predict(X_test)


#  Finding records to be usaed in study

In [6]:


prediction_error = np.where(np.equal(y_pred, y_test) == False)
print("Fields where prediction was wrong")
print(prediction_error)

print("Test vectors for found indexes")
print(X_test[prediction_error])
#
print(f'Predicted value: {y_pred[prediction_error]}')
print(f'Actuall value: {y_test[prediction_error]}')

print("\n ------------------------------------------ \n")

print("Test vector for correct queery")
print(X_test[1532])
print(f'Actuall value: {y_test[1532]}')


Fields where prediction was wrong
(array([1289, 1290]),)
Test vectors for found indexes
[[ 1.         13.          0.6         1.         15.38461538]
 [ 1.         13.          0.6         1.         15.38461538]]
Predicted value: [0. 0.]
Actuall value: [1. 1.]

 ------------------------------------------ 

Test vector for correct queery
[ 0.         28.          0.5         1.          3.57142857]
Actuall value: 0.0


# Determining border vales of fields

In [7]:
fields_dict = ["sql keywords", "params length", "request volume", "specila chars", "values to length"]

init = np.array([[1, 13, 0.6, 1, 15.38461538]])
false_positive = np.array(init, copy=True)

for pos in range(0, len(init[0]), 1):
    false_positive = np.array(init, copy=True)
    prediction = 0
    while prediction == 0:
        false_positive[0,pos] = false_positive[0,pos] +1
        prediction = knn.predict(false_positive)    

    print(f'Border value for {fields_dict[pos]}: {false_positive[0]}')

Border value for sql keywords: [309.          13.           0.6          1.          15.38461538]
Border value for params length: [ 1.         21.          0.6         1.         15.38461538]
Border value for request volume: [ 1.         13.          2.6         1.         15.38461538]
Border value for specila chars: [ 1.         13.          0.6        31.         15.38461538]
Border value for values to length: [ 1.         13.          0.6         1.         26.38461538]


In [8]:
fields_dict = ["sql keywords", "params length", "request volume", "specila chars", "values to length"]

init = np.array([[0, 28, 0.5, 1, 3.57142857]])
false_negative = np.array(init, copy=True)

for pos in range(0, len(init[0]), 1):
    false_negative = np.array(init, copy=True)
    prediction = 0
    while prediction == 0:
        false_negative[0,pos] = false_negative[0,pos] +1
        prediction = knn.predict(false_negative)    

    print(f'Border value for {fields_dict[pos]}: {false_negative[0,pos]}')

Border value for sql keywords: 123.0
Border value for params length: 73.0
Border value for request volume: 6.5
Border value for specila chars: 29.0
Border value for values to length: 13.57142857


# Complex analysis of value changes impacting prediction

In [9]:
fields_dict = ["sql keywords", "params length", "request volume", "specila chars", "values to length"]

init = np.array([[1, 13, 0.6, 1, 15.38461538]])
false_positive = np.array(init, copy=True)
prediction = 0


while prediction == 0:
        false_positive[0,0] = false_positive[0,0] +1
        false_positive[0,1] = false_positive[0,1] +1
        false_positive[0,4] = ((false_positive[0,0] + false_positive[0,3])*100)/ (false_positive[0,1]) 
        prediction = knn.predict(false_positive)    


print(f'Border value for {fields_dict[pos]}: {false_positive[0]}')

Border value for values to length: [ 3.         15.          0.6         1.         26.66666667]


In [10]:
fields_dict = ["sql keywords", "params length", "request volume", "specila chars", "values to length"]

init = np.array([[1, 13, 0.6, 1, 15.38461538]])
false_positive = np.array(init, copy=True)
prediction = 0


while prediction == 0:
        false_positive[0,0] = false_positive[0,0] +1
        false_positive[0,1] = false_positive[0,1] +1
        false_positive[0,4] = ((false_positive[0,0] + false_positive[0,3]))/ (false_positive[0,1]) 
        prediction = knn.predict(false_positive)    


print(f'Border value for {fields_dict[pos]}: {false_positive[0]}')

Border value for values to length: [37.        49.         0.6        1.         0.7755102]


# Conclusion

"values to length" fields is most important in making prediction. Increasing "sql keywords" will result in sql injection dectection because of "values to length" increase. Finding the boarder for max value of "params length" field can help reduce the ratio for field "values to length". After creating sql melicious queery attacker should check if the querry reaches the maximal safe length. If not the querry should be padded with random letters to obtain the maximal length in order to reduce the "values to length" fields value.


Thesis will be tested by conducting series of ezperiments. We will choose some of querries used in learning proces and than modify them in order to avoid the ML detection.

In [20]:
experiment_df = pd.read_csv("sql_df_preprocessed.csv")
experiment_df[experiment_df.duplicated()].drop_duplicates()
experiment_df = experiment_df.where(experiment_df["params_length"] < 25).dropna().reset_index()
experiment_df = experiment_df.where(experiment_df["sql_keyword"] > 1).dropna().reset_index()
experiment_df.sort_values(by = "params_length")

,level_0,index,full_url,parsed_url,hostname,event_time,sql_keyword,params_length,request_volume,special_chars,values_to_length,injection
0,10,35.0,/details.php?prod=5%20ORDER%20BY%201--%20-&typ...,/details.php?prod=5 ORDER BY 1-- -&type=?,192.168.43.6,1.681395e+09,2.0,17.0,97.8,2.0,23.529412,1.0
3,13,38.0,/details.php?prod=5%20ORDER%20BY%206--%20-&typ...,/details.php?prod=5 ORDER BY 6-- -&type=?,192.168.43.6,1.681395e+09,2.0,17.0,97.8,2.0,23.529412,1.0
4,14,39.0,/details.php?prod=5%20ORDER%20BY%204--%20-&typ...,/details.php?prod=5 ORDER BY 4-- -&type=?,192.168.43.6,1.681395e+09,2.0,17.0,97.8,2.0,23.529412,1.0
5,15,40.0,/details.php?prod=5%20ORDER%20BY%205--%20-&typ...,/details.php?prod=5 ORDER BY 5-- -&type=?,192.168.43.6,1.681395e+09,2.0,17.0,97.8,2.0,23.529412,1.0
2,12,37.0,/details.php?prod=5%20ORDER%20BY%2010--%20-&ty...,/details.php?prod=5 ORDER BY 10-- -&type=?,192.168.43.6,1.681395e+09,2.0,18.0,97.8,2.0,22.222222,1.0
17,79,621.0,/details.php?prod=5&type=%3F%20HAVING%209480%3...,/details.php?prod=5&type=? HAVING 9480=9480,192.168.43.6,1.681395e+09,2.0,19.0,97.8,1.0,15.789474,1.0
16,78,620.0,/details.php?prod=5&type=%3F%20HAVING%206971%3...,/details.php?prod=5&type=? HAVING 6971=5614,192.168.43.6,1.681395e+09,2.0,19.0,97.8,1.0,15.789474,1.0
44,402,6117.0,/details.php?type=ASCI&prod=BXPZGEJPXDLJGTT,/details.php?type=ASCI&prod=BXPZGEJPXDLJGTT,192.168.43.6,1.681396e+09,2.0,19.0,1.3,1.0,15.789474,0.0
6,34,258.0,/details.php?prod=5&type=%3F%20OR%20NOT%202276...,/details.php?prod=5&type=? OR NOT 2276=4699,192.168.43.6,1.681395e+09,2.0,19.0,97.8,1.0,15.789474,1.0
7,35,259.0,/details.php?prod=5&type=%3F%20OR%20NOT%205387...,/details.php?prod=5&type=? OR NOT 5387=5387,192.168.43.6,1.681395e+09,2.0,19.0,97.8,1.0,15.789474,1.0


In [30]:
single_record = experiment_df.loc[experiment_df["index"] == 621]
single_record

,level_0,index,full_url,parsed_url,hostname,event_time,sql_keyword,params_length,request_volume,special_chars,values_to_length,injection
17,79,621.0,/details.php?prod=5&type=%3F%20HAVING%209480%3...,/details.php?prod=5&type=? HAVING 9480=9480,192.168.43.6,1.681395e+09,2.0,19.0,97.8,1.0,15.789474,1.0


Found data needs to be refactored (decreess the r"equest volume" and "values to length value" in order to find the max "paramslength" value)

In [49]:
fields_dict = ["sql keywords", "params length", "request volume", "specila chars", "values to length"]

init = single_record.drop(["level_0", "index", "full_url", "parsed_url", "event_time", "hostname", "injection"], axis=1).to_numpy()
print(init)
prediction = 0
attack_vector = np.array(init, copy=True)
attack_vector[0,2] = 1.5
attack_vector[0,4] = 14

while prediction == 0:
        attack_vector[0,1] = attack_vector[0,1] +1
        prediction = knn.predict(attack_vector)    

print(f'Border value for params length: {attack_vector[0]}')

[[ 2.         19.         97.8         1.         15.78947368]]
Border value for params length: [ 2.  22.   1.5  1.  14. ]


We change the url by padding adding additional a letters (extra 30 a letters)

**/details.php?prod=5aaaaaaaaaaaaaaaaaaaaaaaaaaaaaa&type=? HAVING 9480=9480**

alternatively we can change compered numbers:

**/details.php?prod=5&type=? HAVING 9000000000090000=9000000000090000**

In [50]:
fields_dict = ["sql keywords", "params length", "request volume", "specila chars", "values to length"]

init = np.array([[2, 49, 0.6, 1, 6.12244]])
print(init)
attack_vector = np.array(init, copy=True)

prediction = knn.predict(attack_vector) 
prediction  

[[ 2.      49.       0.6      1.       6.12244]]


array([0.])